# Imports

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torchvision import transforms

from random import randrange
import os
import pandas as pd
import numpy as np
from sklearn.utils import resample

import cv2 as cv
from PIL import Image
import PIL.ImageOps    
import matplotlib.pyplot as plt
from skimage.transform import resize

from torch.utils.tensorboard import SummaryWriter
import torchmetrics

import warnings
warnings.filterwarnings("ignore")
import seaborn as sns
from tqdm import tqdm
from sklearn.model_selection import train_test_split

c:\Users\micha\Envs\UKSH\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Globals


In [2]:
dev = "cuda:0" if torch.cuda.is_available() else "cpu" 
device = torch.device(dev) 

size = (150,150)

path = os.path.abspath(os.getcwd())

# Pre Processing
### Calculating mean and std of images

In [3]:
imgs = []

convert = transforms.Compose([   

    transforms.Grayscale(),
    # resize
    transforms.Resize(size),
    # to-tensor
    transforms.ToTensor(),

    
])

for filename in os.listdir("testimages"):
    img = Image.open(f"testimages/{filename}").convert('RGB')
    img = convert(img)
    imgs.append(img)
imgs = torch.stack(imgs)

imgs_mean = imgs.mean()
imgs_std = imgs.std()

In [4]:
transform = transforms.Compose([

    transforms.Grayscale(),
    # resize
    transforms.Resize(size),
    # to-tensor
    transforms.ToTensor(),
    # normalize
    transforms.Normalize((imgs_mean), (imgs_std))
])

# Training NN

In [5]:
class HazelNet(nn.Module):
    """Class for instanciating the NN

    Args:
        nn (nn.Module): super class to inherit from in pytorch
    """

    def __init__(self):
        """ Cosntructor for initialization
        """
        super(HazelNet, self).__init__()
        self.resnet = models.resnet18(pretrained=True)
  
        # over-write the first conv layer to be able to read images
        # as resnet18 reads (3,x,x) where 3 is RGB channels
        # whereas MNIST has (1,x,x) where 1 is a gray-scale channel
        self.resnet.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)

        self.fc_in_features = self.resnet.fc.in_features 
        # remove the last layer of resnet18 (linear layer which is before avgpool layer)
        self.resnet = torch.nn.Sequential(*(list(self.resnet.children())[:-1]))

        # add linear layers to compare between the features of the two images
        self.fc = nn.Sequential(
            nn.Linear(self.fc_in_features, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout2d(p=0.5),
            nn.Linear(4096, 2048),
            nn.ReLU(inplace=True),
            nn.Dropout2d(p=0.5),
            nn.Linear(2048, 1024),
            nn.ReLU(inplace=True),
            nn.Dropout2d(p=0.5),
            nn.Linear(1024, 128),
            )        
       
        # initialize the weights
        self.resnet.apply(self.init_weights)
        self.fc.apply(self.init_weights)   
        
    def init_weights(self, m):
        """Function for weight init

        Args:
            m (module): module to use for init
        """
        if isinstance(m, nn.Linear):
            torch.nn.init.xavier_uniform(m.weight)
            m.bias.data.fill_(0.01)
    
    def forward_once(self, inputs):
        """Helper function for forward path

        Args:
            inputs (tensor): input tensor

        Returns:
            tensor: output tensor
        """
        output = self.resnet(inputs)
        output = output.view(output.size()[0], -1)
        output = self.fc(output)
        return output
    
    def distance_layer(self, vec1, vec2):
        """Function for calculating the cosine similarity between two tensors

        Args:
            vec1 (tensor): tensor for template images
            vec2 (tensor): tensor for images to compare with

        Returns:
            tensor: tensor containing the calculated similarity as float
        """
        cos = torch.nn.CosineSimilarity()
        similarity = cos(vec1, vec2) 
        return similarity

    def forward(self, template, img):
        """Main function for forward path

        Args:
            template (tensor): tensor of template images
            img (tensor): tensor of images to compare

        Returns:
            tensor: tensor containing the calculated similarity as float
        """
        output1 = self.forward_once(template)
        output2 = self.forward_once(img)
        output = self.distance_layer(output1,output2)
 
        return output

    def readImg_url (self, url1, url2, iswhite = False, plot = False):
        """Function for reading images into processable tensors. Can draw a picture of processed images

        Args:
            url1 (string): url to template image
            url2 (string): url to image for comparison
            iswhite (bool, optional): inverts image colors if set to true. Defaults to False.
            plot (bool, optional): plots imported images if set to true. Defaults to False.

        Returns:
            tensor, tensor: two tensors containing the processed images ready for prediction
        """
        # invert white and black if image is on white background
        if iswhite:
            realim1 = cv.bitwise_not(cv.imread(url1,0)).astype(np.float32)
            realim2 = cv.bitwise_not(cv.imread(url2,0)).astype(np.float32)
        else: 
            realim1 = cv.imread(url1,0).astype(np.float32)
            realim2 = cv.imread(url2,0).astype(np.float32)

        realim1 =  cv.resize(realim1, imgsize)
        realim2 =  cv.resize(realim2, imgsize)
        template = torch.tensor(realim1).unsqueeze(0).unsqueeze(0).to(device)
        img = torch.tensor(realim2.astype(np.float32)).unsqueeze(0).unsqueeze(0).to(device)

        if plot:
            fig, ax = plt.subplots(1,2)
            ax[0].imshow(realim1)
            ax[1].imshow(realim2)
        return template, img        

In [6]:
model = HazelNet().to(device)
#torch.save(model.state_dict(), f"test")
model.load_state_dict(torch.load(f"bestmodel_area"))

<All keys matched successfully>

In [3]:
import pandas as pd
pd.read_csv("Flask-Tests/logging")

,img,template,label,pred,HazelArea
0,images\lost.png,images\lost.png,0.6,0.0,NaN
1,images\lost.png,images\lost.png,576.0,0.0,NaN
2,images\lost.png,images\lost.png,0.6,0.0,NaN
3,images\lost.png,images\lost.png,0.6,0.0,NaN
4,images\lost.png,images\lost.png,0.6,0.0,NaN
5,images\lost.png,images\lost.png,0.6,0.0,NaN
6,images\lost.png,images\lost.png,0.7,0.0,NaN
7,images\lost.png,images\lost.png,0.6,1.0,NaN
8,images\lost.png,images\lost.png,0.6,1.0,NaN
9,images\lost.png,images\lost.png,0.6,1.0,NaN
